# Talk to your App

In majority of the examples, LLM Usage is shown as a chat-bot or personal assistant. 

A more powerful usage of Agentic-AI are when they interact with the system in the background and enrich it or take autonomous actions. This is conceptually similar to streaming analytics - with analytics being processed by Agents powered by LLMs.


In [2]:
import openai
import re
import httpx
import os
import rich
import json
from openai import OpenAI
from agents import Agent, ModelSettings, function_tool,Runner,AsyncOpenAI,OpenAIChatCompletionsModel
from dotenv import load_dotenv, find_dotenv
from rich.pretty import pprint
from rich import print

api_key = "sk-dummy_key"
os.environ["OPENAI_API_KEY"] = api_key

model = "llama3.2:3b-instruct-fp16" 
#model = "granite3.2:8b" 
base_url = "http://localhost:11434/v1/"

model = OpenAIChatCompletionsModel( 
    model=model,
    openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1",api_key=api_key),
)

print("[green] Model setup[/green]")

 Model setup

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-dummy_key. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-dummy_key. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-dummy_key. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-dummy_key. You can find your API key at https://platform.ope

## Agents participating in a Trouble shooting Workflow

The investigation into five specialized “agents” that work together like a miniature on-call team.
1. First, the Dependency Identifier agent figures out which backend services power the shopping cart.
1. Next, the Change Tracker agent looks at those services to see what code or configuration changes went out recently.
1. Then the Error Finder agent scans their logs to pull out any real-time error messages.
1. The Debugger agent takes those three fact-sets—dependencies, changes, and errors—and reasons through them step by step to deliver a clear, human-friendly diagnosis of the most likely root cause.
1. Verification Agent whose sole job is to audit each diagnosis before a human being will act on it


In [3]:
@function_tool
def get_dependency(service:str) ->list[str]:
    dep_service=["ProductCatalogService","CheckoutService","UserProfileService"] 
    return dep_service

did_agent = Agent(
    name="DependencyIdentifier Agent",
    instructions=(
        "An incident will be passed on.\n"
        "From that, firstly identify the affected service name only.\n"
        "Next, identify what are the service dependencies for that service.\n"
        "Just return all service names in a comma separated format like a python list[str]. Also include the affected service.\n"
        "And nothing else"
    ),
    model=model,
    tools=[get_dependency],
    model_settings=ModelSettings(temperature=0.0),
)

@function_tool
def get_changelog(service:list) ->list[str]:
    change_log=["ProductCatalogService changed","CheckoutService changed"]
    return change_log

change_agent = Agent(
    name="ChangeLog Agent",
    instructions=(
        "An array of service names will be passed on - like ['ProductCatalogService','CheckoutService','..'].\n"
        "Identify what has changed with these services and return them.\n"
        "Use available tools for this and do not generate data on our own.\n"
        "Just return all changes in a comma separated format like a python list[str].\n"
        "Do not return duplicate changes"
    ),
    model= model,
    tools=[get_changelog],
    model_settings=ModelSettings(temperature=0.0),
)

@function_tool
def get_errorlog(service:list) ->list[str]:
    error_log=["ProductCatalogService is responding slowly"]
    return error_log

error_agent = Agent(
    name="Error Log Agent",
    instructions=(
        "An array of service names will be passed on - like ['ProductCatalogService','CheckoutService','..']. \n"
        "Note that all services may not have error messages and it is unlikely that same message appear in logs of all services. \n"
        "Use available tools for this and do not generate data on our own.\n"
        "The error messages will have service names in the messages. \n"
        "Identify which services have what kind of errors"
    ),
    model=model,
    tools=[get_errorlog],
    model_settings=ModelSettings(temperature=0.0),
)

debugger_agent = Agent(
    name="Debugger Agent",
    instructions=(
        "You will be given:\n"
        "1. Incident details.\n"
        "2. Services that could have been root cause of the problem.\n"
        "3. Services that were changed in the time interval.\n"
        "4. Services that had errors in the logs.\n"
        "Based on the above, loigically think through and conclude the most likely reason for this problem. \n"
        "Please lay down your thought process clearly that led you to the conclusion. "
    ),
    model= model,
    model_settings=ModelSettings(temperature=0.0),
)
print("[green] Agents setup[/green]")

 Agents setup

### Now let us add a new agent
We add a Verification Agent whose sole job is to audit each diagnosis before you act on it. In practice this agent will:

- Read the incident summary, the list of services, and the debugger’s reasoning
- Check for mismatches or missing facts (e.g., a service name dropped or an error overlooked)
- Flag any inconsistencies or confirm “All clear”

By doing so, we get an extra safety net that catches accidental oversights or AI hallucinations. 

In [4]:
verification_agent = Agent(
    name="Verification Agent",
    instructions=(
        "You’ll be given four parts:\n"
        "1) The incident description\n"
        "2) A list of services\n"
        "3) The debugger agent’s full reasoning and conclusion\n\n"
        "Check for any of these issues:\n"
        " • References to services not in the original list\n"
        " • Conclusions that contradict the provided errors/changes\n"
        " • Missing any service that clearly had errors or changes\n\n"
        "If everything is consistent, reply “Consistent”. Otherwise, list the problems."
    ),
    model=model,
    model_settings=ModelSettings(temperature=0.0),
)
print("[green] Verification Agent setup[/green]")

 Verification Agent setup

### Orchestrating this workflow

In [5]:
import asyncio
async def orchestrate(input):
    # Call the intermediate agents to gather the facts
    # These all use tools heavily
    dep_result = await Runner.run(did_agent,input)
    change_result = await Runner.run(change_agent, dep_result.final_output)
    error_result = await Runner.run(error_agent, dep_result.final_output)

    services = dep_result.final_output               # e.g. ["foo","bar","baz"]
    changes  = change_result.final_output             # e.g. ["foo changed","bar changed"]
    errors   = error_result.final_output              # e.g. ["foo is responding slowly"]

    # Build a single prompt string:
    message = (
        "Incident details: " + input + "\n"
        "Affected services: " + services + "\n"
        "Changes detected: " + changes + "\n"
        "Error logs: " + errors + "\n"
        "Based on the above, logically think through and conclude the most likely reason for this problem. "
        "Please lay down your thought process clearly that led you to the conclusion."
    )
    print("\n")
    print("Input to the Deubgger Agent: ")
    print("-----------------------------")
    print(message)
    print("\n")
    # Invoke it:
    debugger_result = await Runner.run(debugger_agent, message)
    print("=== Debugger Thought Process & Conclusion ===")
    print(debugger_result.final_output)

    # New Section:    
    verification_prompt = (
        "Incident details: " + input + "\n"
        "Affected services: " + services + "\n"
        f"Debugger reasoning:\n{debugger_result.final_output}\n\n"
        "Please check:\n"
        " • Are all referenced services in the original list?\n"
        " • Does the conclusion contradict any errors/changes?\n"
        "If everything is consistent, just say “Consistent” and nothing else. "
        "Otherwise list the issues you see."
    )
    
    print("\n")
    print("Input to the Verification Agent: ")
    print("---------------------------------")
    print(verification_prompt)
    print("\n")

    verification_result = await Runner.run(verification_agent, verification_prompt)
    #pprint(verification_result.final_output)
    return verification_result.final_output

print("[green] Agent orchestration setup[/green]")

 Agent orchestration setup

### Calling the orchestration function

In [6]:
input = "Incident: ShoppingCart response time has increased to 10 sec"
diagnosis = await orchestrate(input)
print("=============================================")
print("=== Verifier Thought Process & Conclusion ===")
print("=============================================")
print(diagnosis)

Input to the Deubgger Agent:

-----------------------------

Incident details: Incident: ShoppingCart response time has increased to 10 sec
Affected services: The affected service is ShoppingCart. The dependencies for this service are 
ProductCatalogService, CheckoutService, and UserProfileService.
Changes detected: The changes to the ShoppingCart service include updates from ProductCatalogService and 
CheckoutService. These services have undergone changes, which are reflected in the ShoppingCart service.
Error logs: It appears that the ProductCatalogService is experiencing slow response times. This could be causing 
issues with the ShoppingCart service, which relies on it.

The CheckoutService and UserProfileService do not have any error messages in their logs at this time. 

Please note that you may want to investigate further into the cause of the slow response times on the 
ProductCatalogService to ensure it does not become a recurring issue.
Based on the above, logically think through and conclude the most likely reason for this problem. Please lay down 
your thought process clearly that led you to the conclusion.

=== Debugger Thought Process & Conclusion ===

Here's my thought process:

**Step 1: Identify the affected service**
The incident is affecting the ShoppingCart service, which has increased response times to 10 seconds.

**Step 2: Analyze dependencies**
The ShoppingCart service depends on three other services: ProductCatalogService, CheckoutService, and 
UserProfileService. This means that any issues with these dependent services could be contributing to the problem.

**Step 3: Investigate changes detected**
Changes were detected in the ShoppingCart service, specifically updates from ProductCatalogService and 
CheckoutService. These changes are likely related to the incident.

**Step 4: Examine error logs**
Error logs indicate slow response times for the ProductCatalogService, which is one of the dependent services of 
ShoppingCart.

**Step 5: Eliminate unlikely causes**
The CheckoutService and UserProfileService do not have any error messages in their logs, making it less likely that
they are causing the issue. However, this does not rule out the possibility that these services could still be 
contributing to the problem indirectly.

**Step 6: Draw conclusions**
Based on the analysis, I conclude that the most likely reason for the increased response time of the ShoppingCart 
service is related to issues with its dependent services, specifically ProductCatalogService. The slow response 
times in ProductCatalogService's logs suggest a possible cause for the problem.

**Conclusion**: The most likely root cause of the incident is an issue with the ProductCatalogService, which is 
causing slow response times and indirectly affecting the ShoppingCart service due to their dependency relationship.

**Recommendation**: Further investigation should be conducted into the cause of the slow response times on the 
ProductCatalogService to ensure it does not become a recurring issue.

...

Incident details: Incident: ShoppingCart response time has increased to 10 sec
Affected services: The affected service is ShoppingCart. The dependencies for this service are 
ProductCatalogService, CheckoutService, and UserProfileService.
Debugger reasoning:
Here's my thought process:

**Step 1: Identify the affected service**
The incident is affecting the ShoppingCart service, which has increased response times to 10 seconds.

**Step 2: Analyze dependencies**
The ShoppingCart service depends on three other services: ProductCatalogService, CheckoutService, and 
UserProfileService. This means that any issues with these dependent services could be contributing to the problem.

**Step 3: Investigate changes detected**
Changes were detected in the ShoppingCart service, specifically updates from ProductCatalogService and 
CheckoutService. These changes are likely related to the incident.

**Step 4: Examine error logs**
Error logs indicate slow response times for the ProductCatalogService, which is one of the dependent services of 
ShoppingCart.

**Step 5: Eliminate unlikely causes**
The CheckoutService and UserProfileService do not have any error messages in their logs, making it less likely that
they are causing the issue. However, this does not rule out the possibility that these services could still be 
contributing to the problem indirectly.

**Step 6: Draw conclusions**
Based on the analysis, I conclude that the most likely reason for the increased response time of the ShoppingCart 
service is related to issues with its dependent services, specifically ProductCatalogService. The slow response 
times in ProductCatalogService's logs suggest a possible cause for the problem.

**Conclusion**: The most likely root cause of the incident is an issue with the ProductCatalogService, which is 
causing slow response times and indirectly affecting the ShoppingCart service due to their dependency relationship.

**Recommendation**: Further investigation should be conducted into the cause of the slow response times on the 
ProductCatalogService to ensure it does not become a recurring issue.

Please check:
 • Are all referenced services in the original list?
 • Does the conclusion contradict any errors/changes?
If everything is consistent, just say “Consistent” and nothing else. Otherwise list the issues you see.

=============================================

=== Verifier Thought Process & Conclusion ===

=============================================

Consistent

# Going into Production

We have seen now how a so called output by AI can be cross checked. This is widely used pattern in Agentic workflows.

One other important issue - how does the system Learn ?

1. Let us say Agentic application gives a certain resolution to an incident.
1. And the engineer verifies it to be correct.
1. Or the engineer verifies it not be correct and knows the correct solution.

How can we enhance our agentic application with this.

## Learning
### Integrate Human Feedback
- Whenever the Verification Agent flags a problem, route the case to an engineer for review.
- Provide a simple thumbs-up/thumbs-down or rating interface. Feed that rating back into your store.
- Also think of allowing general text entry field allowing engineer to enter what was done, if it was a thumbs down

### Capture and store outcomes
- Capture the above data and store it in a database

### Feedback the data
- Provide this data as an additional context to the debugger agent (or perhaps add another agent) which looks at this data and fine tunes the recommendation.

### Continuously refine your agents
- Periodically pull the best-rated incident examples (and their human-approved diagnoses) to create few-shot prompts or even fine-tune a custom model.
- Update agent instructions based on common failure modes (e.g. “always double-check inventory data”).

Some of the other next steps could be -

## Build visibility and dashboards
- Surface the agents’ findings and verification results in a team dashboard—showing average time to diagnosis, verification pass rates, and automation success rates.
- Use that data to spot gaps (e.g. Services that consistently fool the Debugger) and add new special-purpose agents.

## Hook into real incident streams
- Connect the orchestration function to the monitoring/alerting system (e.g. Prometheus Alertmanager, CloudWatch Alarms, PagerDuty webhooks). Therefore every time an alert fires for “shopping-cart latency,” the agents automatically kick off the dependency→change→error→debug→verify chain.

## Future State
- Let us imagine a state in future where there may be many agents that can gather data. In the above example it was confined to 3:
  - discovering service dependency,
  - looking at change log and
  - looking at application logs.
Let us say there are agents for metrics, anomaly detection, cluster health (for the cluster on which the service is running on etc).
- In that case, the incident could go to a `Planner agent` that decides to breakdown the troubleshooting into steps and call agents for each steps.
- And then hand over the summary to the Debugger Agent and Verifier agent as shown above.